In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from pathlib import Path
import os.path
import glob
import matplotlib.image as mpimg
import seaborn as sns
import itertools
import json
from PIL import Image 
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.models import Model
from keras.layers import TimeDistributed, LSTM, Input
from keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout, GlobalAveragePooling2D

In [2]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#Uploading the ROIs
!unzip gdrive/My\Drive/ROI.zip

Streaming output truncated to the last 5000 lines.
  inflating: ROI/F069_T3/F069_T3_leftcheek414.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek423.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek425.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek429.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek434.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek436.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek440.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek448.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek449.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek45.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek458.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek460.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek464.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek49.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek502.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek514.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek515.jpg  
  inflating: ROI/F069_T3/F069_T3_leftcheek518.jpg  
  inflating: RO

In [4]:
#Pre-processing the input
def load_dataset(df, m):
    paths = df['Filepath']
    images_dataset = []
    for i in range(len(paths)):
        root = paths[i]
        img = Image.open(root)
        if img.mode == 'RGBA':
            alpha = img.split()[3]
            bgmask = alpha.point(lambda x: 255-x)
            img = img.convert('RGB')
            img.paste((255,255,255), None, bgmask)
        img = img.resize((m, m), Image.ANTIALIAS)
        img = np.array(img)
        means = img.mean(axis=(0,1), dtype='float32')
        stds = img.std(axis=(0,1), dtype='float32')
        # per-channel standardization of pixels
        img = (img - means) / stds            
        images_dataset.append(img)
    return images_dataset

In [5]:
#uploading training set
train_leftcheek =  load_dataset(df = pd.read_csv('/content/gdrive/MyDrive/Left_cheek_BP_train.csv'), m = 32)
train_rightcheek =  load_dataset(df = pd.read_csv('/content/gdrive/MyDrive/Right_cheek_BP_train.csv'), m = 32)
train_leftcheek= np.array(train_leftcheek, dtype= np.uint8).reshape(-1,25,32,32,3)
train_rightcheek= np.array(train_rightcheek,dtype= np.uint8).reshape(-1,25,32,32,3)

In [9]:
#uploading validation set
val_leftcheek =  load_dataset(df = pd.read_csv('/content/gdrive/MyDrive/Left_cheek_BP_val.csv'), m = 32)
val_rightcheek =  load_dataset(df = pd.read_csv('/content/gdrive/MyDrive/Right_cheek_BP_val.csv'), m = 32)
val_leftcheek= np.array(val_leftcheek, dtype= np.uint8).reshape(-1,25,32,32,3)
val_rightcheek= np.array(val_rightcheek,dtype= np.uint8).reshape(-1,25,32,32,3)

In [10]:
#uploading ground truth
df_train = pd.read_csv('/content/gdrive/MyDrive/Ground_Truth_dias_train.csv')
y_train = np.array(df_train['0']).reshape(-1,1)
df_val = pd.read_csv('/content/gdrive/MyDrive/Ground_Truth_dias_val.csv')
y_val = np.array(df_val['0']).reshape(-1,1)

In [ ]:
#construction of the model
res1= EfficientNetB4(include_top=False,input_shape =(32,32,3), weights='imagenet')
res2= EfficientNetB4(include_top=False,input_shape =(32,32,3), weights='imagenet')
input_pic_leftcheek = Input((25,32,32,3))
E2 = res1
for layer in E2.layers[0:194]:
    layer.trainable = False


output2 = GlobalAveragePooling2D()(E2.output)
cnn2 = Model(inputs=E2.input, outputs=output2)
encoded_frames2 = TimeDistributed(cnn2)(input_pic_leftcheek)
lstm2 = LSTM(25)(encoded_frames2)
y = layers.Dense(64, activation='relu')(lstm2)
y = layers.Dropout(0.2)(y)
y = layers.Dense(32, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(16, activation='relu')(y)
y = Model(inputs=input_pic_leftcheek, outputs=y)


input_pic_rightcheek = Input((25,32,32,3))
E3 = res2
for layer in E3.layers[0:194]:
    layer.trainable = False


output3 = GlobalAveragePooling2D()(E3.output)
cnn3 = Model(inputs=E3.input, outputs=output3)
encoded_frames3 = TimeDistributed(cnn3)(input_pic_rightcheek)
lstm3 = LSTM(25)(encoded_frames3)
z = layers.Dense(64, activation='relu')(lstm3)
z = layers.Dropout(0.2)(z)
z = layers.Dense(32, activation='relu')(z)
z = layers.Dropout(0.2)(z)
z = layers.Dense(16, activation='relu')(z)
z = Model(inputs=input_pic_rightcheek, outputs=z)
combined = layers.concatenate([y.output, z.output])
final = layers.Dense(32, activation="relu")(combined)
final = layers.Dense(1, activation="linear")(final)
model = Model(inputs=[y.input, z.input], outputs=final)

71696384/71686520 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 25, 32, 32,  0           []                               
                                 3)]                                                              
                                                                                                  
 input_4 (InputLayer)           [(None, 25, 32, 32,  0           []                               
                                 3)]                                                              
                                                                                                  
 time_distributed (TimeDistribu  (None, 25, 1792)    17673823    ['input_3[0][0]']                
 ted)                                                                                       

In [ ]:
#training the model

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=opt,
    loss='mse',
    metrics = ['mae']
)

filepath = '/content/gdrive/MyDrive/Model_Dias.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min',
                             save_weights_only=True
                            )
callbacks = [checkpoint]

history = model.fit(x=[train_leftcheek, train_rightcheek],
                    y = y_train ,
                    validation_data=([val_leftcheek,val_rightcheek],y_val),
                    epochs=20,
                    batch_size=8,
                    callbacks=callbacks
)

model_json = model.to_json()
with open("/content/gdrive/MyDrive/Model_Dias.json", "w") as json_file:
    json_file.write(model_json)